In [4]:
import http.client
import json
import pandas as pd

## Données des équipes de chaque principale ligue

In [5]:
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': '52209beef1bc4a4e8b08fbf89163915f' }

connection.request('GET', 'https://api.football-data.org/v2/competitions/FL1/teams', None, headers )
equipes_fr = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/CL/teams', None, headers )
equipes_cl = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/PL/teams', None, headers )
equipes_eng = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/PD/teams', None, headers )
equipes_esp = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/SA/teams', None, headers )
equipes_ita = json.loads(connection.getresponse().read().decode())
connection.request('GET', 'https://api.football-data.org/v2/competitions/PPL/teams', None, headers )
matchs_pt = json.loads(connection.getresponse().read().decode())

### Fusion dans un dataframe

In [6]:
df_fr=pd.DataFrame(equipes_fr['teams'])
df_cl=pd.DataFrame(equipes_cl['teams'])
df_eng=pd.DataFrame(equipes_eng['teams'])
df_esp=pd.DataFrame(equipes_esp['teams'])
df_ita=pd.DataFrame(equipes_ita['teams'])
df_pt=pd.DataFrame(matchs_pt['teams'])


columns=['shortName','name']
fr_names=df_fr[columns]
cl_names=df_cl[columns]
eng_names=df_eng[columns]
esp_names=df_esp[columns]
ita_names=df_ita[columns]
pt_names=df_pt[columns]


df_teams=pd.DataFrame(columns=['shortName','name'])
df_teams=df_teams.append(fr_names,ignore_index=True)
df_teams=df_teams.append(eng_names,ignore_index=True)
df_teams=df_teams.append(esp_names,ignore_index=True)
df_teams=df_teams.append(ita_names,ignore_index=True)
df_teams=df_teams.append(cl_names,ignore_index=True)
df_teams=df_teams.append(pt_names,ignore_index=True)

df_teams=df_teams.drop_duplicates()
df_teams

,shortName,name
0,Toulouse,Toulouse FC
1,Brest,Stade Brestois 29
2,Marseille,Olympique de Marseille
3,Montpellier,Montpellier HSC
4,Lille,Lille OSC
...,...,...
172,Belenenses,Os Belenenses Futebol
173,Marítimo,CS Marítimo
174,Portimonense,Portimonense SC
175,Braga,Sporting Clube de Braga


## Récupération des données du site web

In [8]:
teams_valeur=pd.read_csv("Points_Equipes2020.csv",sep=";",index_col=0)

# Conversion équipes API / équipes site web

In [9]:
def most_frequent(List): 
    return max(set(List), key = List.count) 

In [10]:
def exception(equipe):
    
    # Espagne
    if (equipe=="Barça"):
        equipe="Barcelona"
        return equipe
    if (equipe=="Real Sociedad"):
        equipe="Sociedad"
        return equipe
    
    
    
    # Italie
    if (equipe=="Atleti"):
        equipe="Atlético"
        return equipe

    # France

    if (equipe=="Paris SG"):
        equipe="PSG"
        return equipe
    if (equipe=="St Etienne"):
        equipe="Saint-Étienne"
        return equipe
    if (equipe=="Paris SG"):
        equipe="PSG"
        return equipe
    if (equipe=="Stade Rennais"):
        equipe="Rennes"
        return equipe
    return equipe

In [11]:
def recherche_name(nom,data):
    v_mots=[]
    v=[]
    liste=[]
    mots = nom.split(' ')
    for mot in mots:
    # On évite les mots courts comme "de"
        if(len(str(mot))>3):
            v_mots.append(mot)
            # Récupération des équipes possèdant ces mots
            for v_mot in v_mots:
                x=data[data["Equipe"].str.contains(v_mot) == True]
                v.append(x["Equipe"].values)
            
    for item in v:
        if(len(item)>0):
            liste.append(item[0])
            
    if(len(liste)>0):
        equipe_trouvee=most_frequent(liste)
        good_name_str=equipe_trouvee
        
    else:
        # Cette équipe n'existe pas dans nos données donc surement trop faible
        good_name_str= None
    
    return good_name_str

In [12]:
test=teams_valeur[teams_valeur["Equipe"].str.contains("S") == True]
test

,Equipe,Points
11,PSG,9495
28,ShanghaiSIPG,7706
30,Sevilla,7447
31,RealSociedad,7427
39,Santos,7088
...,...,...
897,ManisaBBSK,166
898,Samsunspor,166
907,Songo,162
923,Shkendija,145


In [13]:
def name_web(equipe,df):
    name=None
    equipe=exception(equipe)
    equipe=equipe.replace("Stade","")
    x=df.loc[df['Equipe']==equipe]
    
    if(x.empty):
        name=recherche_name(equipe,df)
        return name
    
    else:    
        n=x['Equipe'].values
        name=n[0]
        return name


In [14]:
df_teams['name_web']=df_teams['shortName'].apply(lambda x: name_web(x,teams_valeur))

In [15]:
df_teams

,shortName,name,name_web
0,Toulouse,Toulouse FC,Toulouse
1,Brest,Stade Brestois 29,Brest
2,Marseille,Olympique de Marseille,OlympiqueMarseille
3,Montpellier,Montpellier HSC,Montpellier
4,Lille,Lille OSC,Lille
...,...,...,...
172,Belenenses,Os Belenenses Futebol,Belenenses
173,Marítimo,CS Marítimo,Marítimo
174,Portimonense,Portimonense SC,Portimonense
175,Braga,Sporting Clube de Braga,SportingBraga


## Récupération de la puissance

In [16]:
def recup_puissance(equipe,df):
    result=df["Points"].loc[df["Equipe"]==equipe]
    
    if (result.empty):
        return 100  
    else: 
        x=result.values
        if (len(x)>1):
            x=x[0]
        return int(x)

In [17]:
df_final=pd.DataFrame()
df_teams['puissance']=df_teams['name_web'].apply(lambda x: recup_puissance(x,teams_valeur))
df_teams

,shortName,name,name_web,puissance
0,Toulouse,Toulouse FC,Toulouse,1065
1,Brest,Stade Brestois 29,Brest,2156
2,Marseille,Olympique de Marseille,OlympiqueMarseille,4008
3,Montpellier,Montpellier HSC,Montpellier,3029
4,Lille,Lille OSC,Lille,4207
...,...,...,...,...
172,Belenenses,Os Belenenses Futebol,Belenenses,1589
173,Marítimo,CS Marítimo,Marítimo,1672
174,Portimonense,Portimonense SC,Portimonense,1067
175,Braga,Sporting Clube de Braga,SportingBraga,5314


In [18]:
df_teams[0:50]

,shortName,name,name_web,puissance
0,Toulouse,Toulouse FC,Toulouse,1065
1,Brest,Stade Brestois 29,Brest,2156
2,Marseille,Olympique de Marseille,OlympiqueMarseille,4008
3,Montpellier,Montpellier HSC,Montpellier,3029
4,Lille,Lille OSC,Lille,4207
5,Nice,OGC Nice,Nice,2803
6,Olympique Lyon,Olympique Lyonnais,OlympiqueLyonnais,6154
7,PSG,Paris Saint-Germain FC,PSG,9495
8,Bordeaux,FC Girondins de Bordeaux,Bordeaux,2298
9,Saint-Étienne,AS Saint-Étienne,Saint-Étienne,3749


## Enregistrement csv

In [19]:
df_teams.to_csv(r"Teams_puiss.csv", sep=";", encoding='utf-8')